# Export GCS routes to CSV

Read every route JSON in the `cycle_more_bucket/all_routes/` prefix, convert coordinates to WKT `LINESTRING`, and save to a CSV for downstream use.

**Notes**
- Uses `fsspec`/`gcsfs` with your existing GCP auth (`google_default`).
- Expects route files to be GeoJSON/ORS-style with `LineString` or `MultiLineString` geometry.
- Output CSV: `route_coordinates.csv` in this repo.

In [9]:
import json
import re
from pathlib import Path
from typing import Iterable, List, Tuple

import fsspec
import pandas as pd

In [10]:
# Configuration
BUCKET = "cycle_more_bucket"
PREFIX = "all_routes/"  # folder under the bucket to scan
TOKEN = "google_default"  # relies on your gcloud auth
OUTPUT_CSV = "route_coordinates.csv"


In [11]:
def list_route_files(bucket: str, prefix: str, token: str = TOKEN) -> List[str]:
    "Return full GCS URIs for every JSON file under the prefix."
    fs = fsspec.filesystem("gcs", token=token)
    pattern = f"gs://{bucket}/{prefix}**/*.json"
    return sorted(fs.glob(pattern))


def load_route_json(uri: str, token: str = TOKEN) -> dict:
    fs = fsspec.filesystem("gcs", token=token)
    with fs.open(uri, "rb") as f:
        return json.load(f)


def extract_route_id(path: str) -> str:
    "Derive a route id from the filename; falls back to the stem."
    stem = Path(path).stem
    match = re.search(r"(\d+)", stem)
    return match.group(1) if match else stem


In [12]:
def extract_coords(route_json: dict) -> Tuple[List[Tuple[float, float]], List[float]]:
    "Extract (lat, lon) points and elevations from GeoJSON/ORS routes."
    if route_json.get("type") == "FeatureCollection":
        features = route_json.get("features", [])
        if not features:
            raise ValueError("No features found in route JSON")
        geom = features[0].get("geometry", {})
    else:
        geom = route_json.get("geometry", {})

    gtype = geom.get("type")
    coords = geom.get("coordinates", [])

    coord_list: List[Tuple[float, float]] = []
    elev_list: List[float] = []

    if gtype == "LineString":
        for lon, lat, elev in coords:
            coord_list.append((lat, lon))
            elev_list.append(elev)
    elif gtype == "MultiLineString":
        for line in coords:
            for lon, lat, elev in line:
                coord_list.append((lat, lon))
                elev_list.append(elev)
    else:
        raise ValueError(f"Unsupported geometry type: {gtype}")

    if not coord_list:
        raise ValueError("No coordinates extracted")

    return coord_list, elev_list


def coords_to_linestring(coords: Iterable[Tuple[float, float]]) -> str:
    # Geo WKT expects lon lat ordering
    coord_pairs = [f"{lon} {lat}" for lat, lon in coords]
    return "LINESTRING(" + ", ".join(coord_pairs) + ")"


In [ ]:
# Discover route files and convert to LINESTRING
route_files = list_route_files(BUCKET, PREFIX)
print(f"Found {len(route_files)} route files under gs://{BUCKET}/{PREFIX}")

records = []
for uri in route_files:
    try:
        route_json = load_route_json(uri)
        coords, _ = extract_coords(route_json)
        linestring = coords_to_linestring(coords)
        records.append({
            "route_id": extract_route_id(uri),
            "gcs_path": uri,
            "linestring": linestring,
        })
    except Exception as exc:  # keep the loop running even if a file is bad
        print(f"Skipping {uri}: {exc}")

if not records:
    raise RuntimeError("No routes processed; confirm bucket/prefix and credentials")

routes_df = pd.DataFrame(records).sort_values("route_id").reset_index(drop=True)
print(f"Converted {len(routes_df)} routes")

routes_df.to_csv(OUTPUT_CSV, index=False)
print(f"Saved CSV to {OUTPUT_CSV}")


Found 16934 route files under gs://cycle_more_bucket/all_routes/


_request out of retries on exception: Cannot connect to host storage.googleapis.com:443 ssl:default [nodename nor servname provided, or not known]
Traceback (most recent call last):
  File "/Users/eugeneleach/.pyenv/versions/3.12.9/envs/cyclemore/lib/python3.12/site-packages/aiohttp/connector.py", line 1532, in _create_direct_connection
    hosts = await self._resolve_host(host, port, traces=traces)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/eugeneleach/.pyenv/versions/3.12.9/envs/cyclemore/lib/python3.12/site-packages/aiohttp/connector.py", line 1148, in _resolve_host
    return await asyncio.shield(resolved_host_task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/eugeneleach/.pyenv/versions/3.12.9/envs/cyclemore/lib/python3.12/site-packages/aiohttp/connector.py", line 1179, in _resolve_host_with_throttle
    addrs = await self._resolver.resolve(host, port, family=self._family)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

Skipping cycle_more_bucket/all_routes/route_17348338.json: Cannot connect to host storage.googleapis.com:443 ssl:default [nodename nor servname provided, or not known]


_request out of retries on exception: Cannot connect to host storage.googleapis.com:443 ssl:default [nodename nor servname provided, or not known]
Traceback (most recent call last):
  File "/Users/eugeneleach/.pyenv/versions/3.12.9/envs/cyclemore/lib/python3.12/site-packages/aiohttp/connector.py", line 1532, in _create_direct_connection
    hosts = await self._resolve_host(host, port, traces=traces)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/eugeneleach/.pyenv/versions/3.12.9/envs/cyclemore/lib/python3.12/site-packages/aiohttp/connector.py", line 1148, in _resolve_host
    return await asyncio.shield(resolved_host_task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/eugeneleach/.pyenv/versions/3.12.9/envs/cyclemore/lib/python3.12/site-packages/aiohttp/connector.py", line 1179, in _resolve_host_with_throttle
    addrs = await self._resolver.resolve(host, port, family=self._family)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

Skipping cycle_more_bucket/all_routes/route_17348602.json: Cannot connect to host storage.googleapis.com:443 ssl:default [nodename nor servname provided, or not known]


In [ ]:
routes_df.head()